In [2]:
from datetime import datetime
import math

In [3]:
#Julain Day
user_input = input("Enter a date (YYYY-MM-DD): ")
user_date = datetime.strptime(user_input, "%Y-%m-%d").date()
julian_day = user_date.timetuple().tm_yday

#Latitude and Longitude of the test site
print("Enter the coordinates of the test site in the format (Degrees°Minutes')\n")

latitude = input("Latitude ")
degrees, minutes = latitude.split("°")
degrees = int(degrees)
minutes = int(minutes[:-1])
latitude = float(degrees + (minutes / 60))
latitude_radians = math.radians(latitude)

longitude = input("Longitude ")
degrees, minutes = longitude.split("°")
degrees = int(degrees)
minutes = int(minutes[:-1])
longitude = float(degrees + (minutes / 60))
longitude_radians = math.radians(longitude)

#Elevation of the test site
elevation = int(input("Enter elevation of the test site: "))

#Wind speed at 2meters
wind_speed = float(input("Enter wind speed at 2 meters from ground: "))
wind_speed = 5*wind_speed/18

#Temperatures
max_temp = float(input("Enter today's maximum temperature: "))
min_temp = float(input("Enter today's minimum temperature: "))
mean_temp = (max_temp + min_temp)/2

#Relative Humidity
max_RH = int(input("Enter the relative humidty corresponding to maximum temperature: "))
min_RH = int(input("Enter the relative humidty corresponding to minimum temperature: "))
mean_RH = (max_RH + min_RH)/2

#Dew Point Temperature
max_DPT = (4030*(235+max_temp)/(4030-((max_temp+235)*math.log(max_RH/100))))-235
min_DPT = (4030*(235+min_temp)/(4030-((min_temp+235)*math.log(min_RH/100))))-235
mean_DPT = (max_DPT + min_DPT)/2

#Actual Sunshine Hours, measured using Campbell-Stokes Sunshine Recorder
n = float(input("Enter the actual sunshime duration of the day recorded "))

#Rainfall Data
rainfall = float(input("Enter the rainfall of the day: "))

Enter the coordinates of the test site in the format (Degrees°Minutes')



In [4]:
#Constants
lambdaa = 2.45                                                                                                      #Latent Heat of Vapourization
alpha = 0.23                                                                                                        #Albedo / Crop Canopy Coefficient
sigma = 4.903e-9                                                                                                    #Stefan-Bolzmann Constant
Gsc = 0.082                                                                                                         #Solar Constant

In [5]:
PI = math.pi
solar_dec = 0.409 * math.sin((2 * PI * julian_day / 365) - 1.39)                                                    #Solar Declination (in radians)
dr = 1 + 0.033 * math.cos(2 * PI * julian_day / 365)                                                                #Inverse Relative Distance Earth-Sun
ws = math.acos(-math.tan(latitude_radians) * math.tan(solar_dec))                                                   #Sunset/Sunrise hour angle (in radians)
N = 24*ws/PI                                                                                                        #Daylight Hours
sinx = (ws * math.sin(latitude_radians) * math.sin(solar_dec)) + (math.cos(latitude_radians) * math.cos(solar_dec)) #sin(Solar Altitude Angle)
Ra = 24 * 60 * Gsc * dr * sinx / PI                                                                                 #Extraterrestrial Radiation
Rs = (0.25+(0.5*(n/N)))*Ra                                                                                          #Solar Radiation
Rso = (0.75 + (2e-5*elevation))*Ra                                                                                  #Clear-Sky Solar Radiation
f = (1.35*Rs/Rso) - 0.35                                                                                            #Cloudiness Function
ea = 0.6108*(math.exp(17.27*mean_DPT/(mean_DPT+237.3)))                                                             #Actual Vapour Pressure
es = 0.6108*(math.exp(17.27*mean_temp/(mean_temp+237.3)))                                                           #Saturated Vapour Pressure
epsilon = 0.34 - (0.14 * math.sqrt(ea))                                                                             #Apparent Net Clear Sky Emmisivity
delta = 4098*es/(mean_temp+237.3)**2                                                                                #Slope of Saturation Vapour Pressure Curve
Bp = 101.3 * ((1 - (0.0000221843 * elevation)) ** 5.6)                                                              #Barometric Pressure
gamma = 0.00163*Bp/lambdaa                                                                                          #Psychometric Constant
Rns = (1-alpha)*Rs                                                                                                  #Net Shortwave Radiation
Rnl = f*epsilon*sigma*(mean_temp + 273.15)**4                                                                       #Net Longwave Radiation
Rn = Rns - Rnl                                                                                                      #Net Radiation
G = 0                                                                                                               #Soil Heat Flux
numerator = 0.408*delta*(Rn-G) + gamma*wind_speed*900*(es-ea)/(mean_temp+273)
denominator = delta + gamma*(1+(0.34*wind_speed))
ETo = numerator/denominator                                                                                         #Reference Evapotranspiration
print("The calculated Reference Evapotranspiration is",ETo,"mm/day")

The calculated Reference Evapotranspiration is 4.471414050562653 mm/day


In [7]:
#Constants
Kc = 0.6                                                                                                            #Crop Coefficient
RootDepth = 0.3                                                                                                     #Root Depth
FC = 18                                                                                                             #Field Capacity
WP = 5                                                                                                              #Wilting Point
P = 0.5                                                                                                             #Soil Moisture Depletion Fraction
Q = 0.75                                                                                                            #Maximum Allowable Depletion Fraction
TAW = 10*(FC-WP)*RootDepth                                                                                          #Total Available Water
RAW = P*TAW                                                                                                         #Readily Available Water
MAD = Q*TAW                                                                                                         #Maximum Allowable Depletion